# Transformers

In this notebook, we are going to define and implement a Transformer model.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

## Scaled Dot-Product Attention

<img src='assets/1/scaled-dot-product-3.PNG'/> 

In [2]:
# Scaled Dot-Product Attention calculation 
# from the paper (section 3.2.1 Scaled Dot-Product Attention):
# 'We compute the dot products of the query with all keys, divide each by √dk, 
# and apply a softmax function to obtain the weights on the values.'

class ScaledDotProductAttention(nn.Module):
    def __init__(self, mask=False):
        super(ScaledDotProductAttention, self).__init__()
        self.mask=mask
        
    def forward(self, Q, K, V):
        
        # compute dot product of the all query with all keys
        K_transpose = torch.transpose(K, -2, -1)   # transpose on last two dimensions
        dot_products = torch.matmul(Q, K_transpose) 
        
        # divide each by √dk
        d_k = K.shape[-1]    # get length of key vector
        scaled_dot_products = dot_products / np.sqrt(d_k)
        
        # apply a softmax function to obtain weights(attention scores) on values
        weights = F.softmax(scaled_dot_products, dim=-1)
        
        # get weighted values by multiplying values with softmax scores
        weighted_values = torch.matmul(weights, V)
        
        # apply mask to prevent positions from attending to subsequent positions in decoder
        if self.mask==True:
            size = weighted_values.shape
            look_ahead_mask = torch.triu(torch.full(size, float('-inf')), diagonal=1)
            look_ahead_mask[look_ahead_mask == 0] = 1
            weighted_values = weighted_values * look_ahead_mask
        
        return weighted_values


Scaled Dot-product attention is identical to the [Dot-product attention algorithm](https://arxiv.org/pdf/1508.04025.pdf), except for the scaling  factor of 1/√dk.
>For large values of dk, the dot products grow large in magnitude, pushing the softmax function into regions where it has extremely small gradients. To counteract this effect, we scale the dot products by 1/√dk. <br><sub>-from section 3.2.1 Scaled Dot-Product Attention<sub>

**Masked Multi-Head Attention**

<img src='assets/1/masked-attention.PNG' width=50% height=50% /> 

>We need to prevent leftward information flow in the decoder to preserve the auto-regressive property. We **implement this inside of scaled dot-product attention** by masking out (setting to −∞) all values in the input of the softmax which correspond to illegal connections. <br><sub>-from section 3.2.3 Applications of Attention in our Model<sub>

>We also modify the self-attention sub-layer in the decoder stack to prevent positions from attending to subsequent positions. This masking, combined with fact that the output embeddings are offset by one position, ensures that the predictions for position i can depend only on the known outputs at positions less than i. <br><sub>-from section 3.1 Encoder and Decoder Stacks<sub>

Let's do a unit test on Scaled Dot-Product Attention implementation.

<img src='assets/1/scaled-dot-product-matrix.PNG' /> 

In [3]:
# define data, n=3 embeddings(sequence length or number of tokens), ex: [Je, suis, etudiant]
n = 3

# from section 3.2.2 Multi-Head Attention
# d_key = d_value = d_query
d_key = 64 # key dimensionality

# define K,Q and V matrices
K = torch.randn(n, d_key)
Q = torch.randn(n, d_key)
V = torch.randn(n, d_key)

scaled_dot_product = ScaledDotProductAttention()

# apply scaled-dot product attention
attention_scores = scaled_dot_product(Q, K, V)    

assert attention_scores.shape == (n, d_key)
attention_scores.shape 

torch.Size([3, 64])

In [4]:
# unit test for masked scaled dot-product attention 

scaled_dot_product = ScaledDotProductAttention(mask=True)
masked_attn_scores = scaled_dot_product(Q, K, V)

assert masked_attn_scores.shape == (n, d_key)
print(masked_attn_scores.shape)

print(masked_attn_scores[:, :5])

torch.Size([3, 64])
tensor([[-0.2168,    -inf,     inf,    -inf,     inf],
        [ 0.4870,  1.5015,    -inf,    -inf,    -inf],
        [-0.1390,  1.0056, -1.1646,    -inf,     inf]])


## Attention

<img src='assets/1/attention-3.PNG'/> 

In [5]:
class Attention(nn.Module):
    def __init__(self, d_model=512, d_key=64, mask=False):
        super(Attention, self).__init__()
        
        # define Key, Query and Value weight matrices
        self.WK = nn.Parameter(torch.randn(d_model, d_key))
        self.WQ = nn.Parameter(torch.randn(d_model, d_key))
        self.WV = nn.Parameter(torch.randn(d_model, d_key))
        
        # init scaled dot-product attention
        self.scaled_dot_product = ScaledDotProductAttention(mask)
        
    def forward(self, inputs, encoder_output=None):
        WK, WQ, WV = self.WK, self.WQ, self.WV
        X = inputs if encoder_output is None else encoder_output
        
        # apply linear transformations
        
        # get packed Key, Query and Value matrices
        # by multiplying inputs with K, Q and V weight matrices
        K = torch.matmul(X, WK)
        V = torch.matmul(X, WV)
        Q = torch.matmul(inputs, WQ)
            
        # apply scaled dot-product attention
        attention_scores = self.scaled_dot_product(Q, K, V)
        
        return attention_scores

>An attention function can be described as mapping a query and a set of key-value pairs to an output, where the query, keys, values, and output are all vectors. The output is computed as a weighted sum of the values, where the weight assigned to each value is computed by a compatibility function of the query with the corresponding key. <br><sub>-from section 3.2 Attention<sub>

>In practice, we compute the attention function on a set of queries simultaneously, packed together into a matrix Q. The keys and values are also packed together into matrices K and V. <br><sub>-from section 3.2.1 Scaled Dot-Product Attention<sub>

It can be inferred from the paper that K, Q and V weight matrices are distinct but in practice, in implementation, there is only **one weight matrix** and projected matrices are also one matrix. The reason for this is to implement the model with highly optimized matrix multiplication code.     
We will not implement this way in this notebook because we want the matrices to be distinguishable so that the code could be easier to follow.

<img src='assets/1/weight-impl.PNG' />

In decoder attention layer, we get the linearly projected K and V matrices by considering encoder output.
<img src='assets/1/encoder-out.PNG' width=50% height=50% align='left'/>

Let's do a unit test on Attention implementation.

<img src='assets/1/attention-matrix.PNG'/>

In [6]:
# from section 3.1 Encoder and Decoder Stacks
d_model = 512    # embedding dimensionsionality

b = 2   # batch size

# define input
# b, n, d_model -> batch size, sequence length(number of embeddings), embedding dimensionality
X = torch.randn(b, n, d_model)

# init attention 
attention = Attention(d_model, d_key)

# compute attention scores
attention_scores = attention(X)

assert attention_scores.shape == (b, n, d_key)
attention_scores.shape

torch.Size([2, 3, 64])

## Multi-Head Attention

<img src='assets/1/multi-head-attention-3.PNG' width=75% height=75%/>

In [7]:
# Multi-Head Attention consists of several attention layers running in parallel.       
class MultiHeadAttention(nn.Module):
    def __init__(self, n_head=8, d_model=512, mask=False):
        super(MultiHeadAttention, self).__init__()
        self.mask = mask
        
        # number of heads
        self.h = n_head
        
        # dimensionality of key vector
        assert d_model % n_head == 0
        d_key = int(d_model / n_head)
        
        # add attention layers to a ModuleList container
        attention_list = [Attention(d_model, d_key, mask) for _ in range(n_head)]
        self.multi_head_attention = nn.ModuleList(attention_list)
        
        # define linear layer 
        self.W = nn.Parameter(torch.randn(n_head*d_key, d_model))
    
    def forward(self, x, encoder_output=None):
        # apply & concat attention layers
        attention_scores = [attention(x, encoder_output) for attention in self.multi_head_attention]
        Z = torch.cat(attention_scores, -1)
        
        # apply linear transformation
        output = torch.matmul(Z, self.W)    # reduce dimensionality
        
        return output
    
# In this work we employ h = 8 parallel attention layers, or heads. 
# For each of these we use d_k = d_v = d_model/h = 64
# - from section 3.2.2 Multi-Head Attention
# We can conclude that d_key is not a hyperparameter
# and that d_model should be divisible by the number of heads.        

Let's do a unit test on Multi-Head Attention implementation.

<img src='assets/1/multi-head-matrix.PNG'/>

In [8]:
# from section 3.2.2 Multi-Head Attention
h = 8 # number of heads

# define input
X = torch.randn(b, n, d_model)

# init multi-head attention 
multi_head_attention = MultiHeadAttention(h, d_model)

# compute multi_head attention score
Z = multi_head_attention(X)

assert Z.shape == (b, n, d_model)
Z.shape

torch.Size([2, 3, 512])

## Feed Forward Neural Network
<img src='assets/1/ffnn.PNG' width=75% height=75%/>


In [9]:
# The dimensionality of input and output is dmodel = 512, 
# and the inner-layer has dimensionality d_ff = 2048
# - from section 3.3 Position-wise Feed-Forward Networks

class FeedForwardNetwork(nn.Module):
    def __init__(self, d_model=512, d_feedforward=2048):
        super(FeedForwardNetwork, self).__init__()
        
        # define the feedforward neural network 
        self.feedforwardnn = nn.Sequential(   
                nn.Linear(d_model, d_feedforward),
                nn.ReLU(),
                nn.Linear(d_feedforward, d_model),
        )
        
    def forward(self, x):
        x = self.feedforwardnn(x)
        return x

In [10]:
# unit test of FeedForward network

# from section 3.3 Position-wise Feed-Forward Network
d_feedforward = 2048

# define input
X = torch.rand(b, n, d_model)

# define feedforward neural network
ffnn = FeedForwardNetwork(d_model, d_feedforward)

# compute ffnn output
output = ffnn(X)

assert output.shape == (b, n, d_model)
output.shape

torch.Size([2, 3, 512])

## Sublayer


<img src='assets/1/sublayer.PNG' width=80% height=80%/> 

In [11]:
# We employ a residual connection around each of the two sub-layers, 
# followed by layer normalization. That is'The output of each sub-layer is
# LayerNorm(x + Sublayer(x)), where Sublayer(x) is
# the function implemented by the sub-layer itself.'
# from section - 3.1 Encoder and Decoder Stacks

# define residual learning block
class Residual_Connection(nn.Module):
    def __init__(self, layer):
        super(Residual_Connection, self).__init__()
        self.layer = layer     # function implemented by the layer

    def forward(self, x):
        f_x = self.layer(x)                    # apply Sublayer(x)
        x = x + f_x                            # x + Sublayer(x)
        return x
    
# To facilitate these residual connections, all sub-layers in the model, as well as the embedding
# layers, produce outputs of dimension dmodel = 512
class Sublayer(nn.Module):
    def __init__(self, layer, d_model=512):
        super(Sublayer, self).__init__()
    
        self.res_connect = Residual_Connection(layer) 
        self.layer_norm = nn.LayerNorm(d_model)  # embedding vector length
        
    def forward(self, x):
        x = self.res_connect(x)
        x = self.layer_norm(x)
        return x

In [12]:
# unit test on sublayer

# define input
X = torch.rand(b, n, d_model)

# init feedforward neural network (with default settings) sublayer
nn_layer = Sublayer(FeedForwardNetwork())

# compute ffnn sublayer output
ffnn_output = nn_layer(X)

# init attention sublayer (with default settings)
attention_layer = Sublayer(MultiHeadAttention())

# compute attention sublayer output
attention_output = attention_layer(X)

assert ffnn_output.shape == (b, n, d_model)
assert attention_output.shape == (b, n, d_model)

print(ffnn_output.shape)
print(attention_output.shape)

# We will not utilize Sublayer module implemented above to observe the flow in Encoder & Decoder.

torch.Size([2, 3, 512])
torch.Size([2, 3, 512])


## Encoder

<img src='assets/1/encoder.PNG' /> 

In [13]:
class Encoder(nn.Module):
    def __init__(self, n_head=8, d_model=512, d_feedforward=2048):
        super(Encoder, self).__init__()
        
        self.attention = MultiHeadAttention(n_head, d_model)
        self.feed_forward = FeedForwardNetwork(d_model, d_feedforward)
        
        self.layer_norm1 = nn.LayerNorm(d_model)
        self.layer_norm2 = nn.LayerNorm(d_model)
        
    def forward(self, x):
        # pass through attention layer
        attention_scores = self.attention(x)
        # apply residual connection & layer normalization
        x = self.layer_norm1(attention_scores + x)  
        
        # pass through feedforward network layer
        scores = self.feed_forward(x)
        # apply residual connection & layer normalization
        x = self.layer_norm2(scores + x)
        
        return x
    
# We will not add any dropout since we will not do any training in this notebook.

In [14]:
# unit test on encoder

# The encoders are all identical in structure (yet they do not share weights)

# define input
X = torch.rand(b, n, d_model)

# init encoder
encoder = Encoder(n_head=8, d_model=512, d_feedforward=2048)

# compute encoder output
output = encoder(X)

assert output.shape == (b, n, d_model)
output.shape

torch.Size([2, 3, 512])

## Decoder

<img src='assets/1/decoder.PNG' /> 

In [15]:
class Decoder(nn.Module):
    def __init__(self, n_head=8, d_model=512, d_feedforward=2048):
        super(Decoder, self).__init__()
        
        # three sub-layers
        # 1. Masked Self-Attention
        # 2. Self-Attention
        # 3. Feedforward Neural Network
        
        self.masked_attention = MultiHeadAttention(n_head, d_model, mask=True)
        self.attention = MultiHeadAttention(n_head, d_model)
        self.feedforward = FeedForwardNetwork(d_model, d_feedforward)
        
        self.layer_norm1 = nn.LayerNorm(d_model)
        self.layer_norm2 = nn.LayerNorm(d_model)
        self.layer_norm3 = nn.LayerNorm(d_model)
        
    def forward(self, x, encoder_output):
        # apply masked attention layer
        masked_attention_scores = self.masked_attention(x)
        # apply residual connection & layer normalization
        x = self.layer_norm1(masked_attention_scores + x)  
        
        # apply attention layer
        attention_scores = self.attention(x, encoder_output)
        x = self.layer_norm2(attention_scores + x)
        
        # apply feedforward network layer
        scores = self.feedforward(x)
        x = self.layer_norm3(scores + x)
        
        return x

In [16]:
# unit test on decoder

# define input for decoder (remember that decoder input is the expected output)
y = torch.rand(b, n, d_model)   
encoder_output = torch.rand(b, n, d_model)

# init decoder
decoder = Decoder(n_head=8, d_model=512, d_feedforward=2048)

# compute decoder output
output = decoder(y, encoder_output)

assert output.shape == (b, n, d_model)
output.shape

torch.Size([2, 3, 512])

## Positional Encoding 

<img src='assets/1/pos-encoding.PNG' /> 

In [17]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model):
        super(PositionalEncoding, self).__init__()
        self.d_model = d_model
        
    def forward(self, x):
        # init positional encoding matrix
        PE = torch.zeros(x.shape[1:])    # exclude batch size
        
        # init positions (sequence, number of tokens)
        position = torch.arange(0, PE.shape[0])
        # init i(dimensions)
        i = torch.arange(0, self.d_model // 2)  # since 2i is -> d, we get i -> d / 2
        
        # division term -> 10000 ^ (2i / d_model) 
        div_term = torch.pow(10000, 2 * i / self.d_model) 
        
        # apply sin to even indices(columns) in the position; 2i
        PE[:, 0::2] = torch.sin(position[:, np.newaxis] / div_term)
        
        # apply cos to odd indices(columns) in the position; 2i + 1
        PE[:, 1::2] = torch.cos(position[:, np.newaxis] / div_term)
        
        return PE

>Since our model contains no recurrence and no convolution, in order for the model to make use of the order of the sequence, we must inject some information about the relative or absolute position of the tokens in the sequence. To this end, we add "positional encodings" to the input embeddings at the bottoms of the encoder and decoder stacks. The positional encodings have the same dimension dmodel as the embeddings, so that the two can be summed. <br><sub>-from section 3.5 Positional Encoding<sub>

<img src='assets/1/pos-encoding-2.PNG' /> 

In [18]:
# unit test for positional encoding 

# define input (assume input is embedded) 
X = torch.randn(b, n, d_model)

# init positional encoding 
pos_encoding = PositionalEncoding(d_model)

# compute and add positional encoding of the embedding inputs 
embeddings = X + pos_encoding(X) 

assert embeddings.shape == (b, n, d_model)
embeddings.shape

torch.Size([2, 3, 512])

## Embedding and Pre-Softmax Linear Layer 

<img src='assets/1/embed.PNG' width=45% height=45% align='left'/> 


>Similarly to other sequence transduction models, we use learned embeddings to convert the input tokens and output tokens to vectors of dimension dmodel. We also use the usual learned linear transformation and softmax function to convert the decoder output to predicted next-token probabilities. In our model, **we share the same weight matrix between the two embedding layers and the pre-softmax linear transformation**. In the embedding layers, we multiply those weights by √dmodel.<br><sub>-from section 3.4 Embeddings and Softmax<sub>


## Transformer Model

<img src='assets/1/transformer-model.PNG' width=80% height=80%/> 

In [19]:
class Transformer(nn.Module):    
    def __init__(self, n_token=37000, n_layer=6, n_head=8, d_model=512, d_ff=2048):
        super(Transformer, self).__init__()
        self.n_token = n_token
        self.n_layer = n_layer
        self.n_head = n_head
        self.d_model = d_model
        self.d_ff = d_ff
        

        # initialize the encoder and decoder stacks
        encoders, decoders = [], []
        
        # stack encoders
        for i in range(n_layer):
            encoder = Encoder(n_head, d_model, d_ff)
            encoders.append(encoder)
        
        # stack decoders
        for i in range(n_layer):
            decoder = Decoder(n_head, d_model, d_ff)
            decoders.append(decoder)
            
        self.encoders = nn.Sequential(*encoders)
        self.decoders = nn.ModuleList(decoders)
        
        # the encoder embedding, and decoder embedding 
        # and decoder pre-softmax transformation(linear layer)
        # share embeddings weights
        #
        # define embedding layer
        self.embedding = nn.Embedding(n_token, d_model)   # num_embeddings, embedding_dim
        
        # define positional encoding layer 
        self.pos_encoding = PositionalEncoding(d_model)
        
        # define softmax layer
        self.softmax = nn.Softmax(dim=-1)
        
    def forward(self, x, y):
        # embed inputs & outputs and get positional encoding
        x = self.embedding(x) * np.sqrt(self.d_model)
        x = x + self.pos_encoding(x)
        
        y = self.embedding(y) * np.sqrt(self.d_model)
        y = y + self.pos_encoding(y)
        
        # pass through encoders and get the output
        encoder_output = self.encoders(x)
        
        # pass through decoders
        decoder_output = y
        for decoder in self.decoders:
            decoder_output = decoder(decoder_output, encoder_output)
        
        # pass through linear layer
        # 
        # a linear layer multiplies the input with a transpose of the weight matrix
        scores = torch.matmul(
            decoder_output, torch.transpose(self.embedding.weight, -2, -1)
        )
        
        # pass through softmax
        output = self.softmax(scores)
        
        return output

<img src='assets/1/transformer-comp.PNG' width=40% height=40% align="left"/>

In [20]:
# unit test for transformer model

# -> we define decoders as ModuleList 
# since Sequential container does not accept multiple inputs for forward method

# from section 3.1 Encoder and Decoder Stacks
# number of stacked layers of encoders & decoders
n_layer = 6

# from section 5.1 Training Data and Batching
n_token = 37000   # vocabulary size

# define input, 
# since embedding expects token indices, init as LongTensor(torch.int64) 
X = torch.randint(0, n_token, (b, n))

# define output 
y = torch.randint(0, n_token, (b, n))

# shift to right
# to ensure that predictions at a specific position "i" can only depend at positions less than i
y = torch.roll(y, 1)

# init transformer
transformer = Transformer(n_token=37000, n_layer=6, n_head=8, d_model=512, d_ff=2048)

# compute output
output = transformer(X, y)

assert output.shape == (b, n, n_token)
output.shape

torch.Size([2, 3, 37000])

---
**Parallelization of Multi-Head Attention in Code**

Multi-Head attention works with Attention layers running in parallel. However, currently there does  not exist an official implementation in PyTorch for parallel modules. Developers are using third-party libraries in their code to utilize parallelism. <br>
Executing the code on GPU avoids sequential execution since underlying execution is asynchronous. **If we have multiple GPUs in our system (and don’t use data parallel), we could execute different modules on each device and concatenate the result back on a single device.**

---
**Key, Value and Query**

The key/value/query concept is analogous to retrieval systems. For example, when we search for videos on Youtube, the search engine will map our query (text in the search bar) against a set of keys (video title, description, etc.) associated with candidate videos in their database, then present us the best matched videos (values). The dot product can be considered as defining some similarity between the text in search bar (query) and titles in the database (key).

---
**Transformers Success**

>We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine translation tasks show these models to
be superior in quality while being more parallelizable and requiring significantly
less time to train. <br><sub>-from section Abstract<sub><br>

>Motivating our use of self-attention we consider three desiderata.
One is the total computational complexity per layer. Another is the amount of computation that can
be parallelized, as measured by the minimum number of sequential operations required.
The third is the path length between long-range dependencies in the network.<br><sub>-from section 4 Why Self-Attention<sub><br>
    
The major success of the Transformer model is that it is parallelizable. Formerly, with recurrent models, we were not able to parallelize since each time step outputs of sequences were depending on previous time step outputs. With Transformer, we can feed the data to the model at the same time and can obtain output. This leads to faster training compared to other RNN & LSTM variant sequence to sequence recurrent models. Transformer model is considered as the state-of-art for language models and NLP.

**Transformers Weakness**<br>

Transformer model requires a significant amount of data to train effectively. Similar to other deep learning models, it is prone to overfitting if trained with small amount of data. Another drawback is, the high computational cost required to run a Transformer effectively. The self-attention mechanism requires a lot of computation, making it more resource-intensive than other deep learning models.  
    
---

### Quick Recap of Familiar Components

<img src='assets/1/components.PNG'/> 

### Final Notes

The implementation here is a very basic representation of how Transformers work. There are a lot of lacking components that are avoided here to preserve the simplicity of the code. There are also a lot of improvements that can be made in the implementation here, and calculations can be further optimized. <br>

The training part and the data handling part is not discussed.

### Resources

1. [Illustrated Transformer](https://jalammar.github.io/illustrated-transformer/) - Blog Post
2. [Attention is All You Need](https://arxiv.org/pdf/1706.03762.pdf) - Paper
3. [Transformers for Beginners | What are they and how do they work](https://www.youtube.com/watch?v=_UVfwBqcnbM&t=4s) - Video
4. [Implementing a Transformer from Scratch](https://towardsdatascience.com/7-things-you-didnt-know-about-the-transformer-a70d93ced6b2) - Blog Post
5. [Transformer’s Encoder-Decoder: Let’s Understand The Model Architecture](https://kikaben.com/transformers-encoder-decoder/) - Blog Post
6. [A Gentle Introduction to Positional Encoding in Transformer Models, Part 1](https://machinelearningmastery.com/a-gentle-introduction-to-positional-encoding-in-transformer-models-part-1/#:~:text=Positional%20encoding%20describes%20the%20location,item's%20position%20in%20transformer%20models.) - Blog Post
7. [Visual Guide to Transformer Neural Networks - (Episode 1) Position Embeddings](https://www.youtube.com/watch?v=dichIcUZfOw) - Video
8. [Transformer Model Inference Visualization](https://miro.medium.com/v2/resize:fit:1100/1*HDqn93FsA93o2z4bTwqUVQ.gif) - GIF
9. [What exactly are keys, queries, and values in attention mechanisms?](https://stats.stackexchange.com/questions/421935/what-exactly-are-keys-queries-and-values-in-attention-mechanisms)